<a href="https://colab.research.google.com/github/tyemmett/portfolio/blob/main/Populationdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial data inspection on city population data
___

In [1]:
import numpy as np
import pandas as pd
import re
import us
import plotly.graph_objects as go

#mount google drive
from google.colab import drive
drive.mount('/content/drive')

#create blank DataFrame for city population data
city_pop_df = pd.DataFrame()

#load data from csv
csv_city_pop = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Population data/citypopulation_fm.csv')
city_pop_df = pd.DataFrame(csv_city_pop)
#inspect city poulation df
city_pop_df.info()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28529 entries, 0 to 28528
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Country or Area  28528 non-null  object
 1   Year             28528 non-null  object
 2   Area             28365 non-null  object
 3   Sex              28365 non-null  object
 4   City             28365 non-null  object
 5   City type        28365 non-null  object
 6   Record Type      28365 non-null  object
 7   Reliability      28365 non-null  object
 8   Source Year      28365 non-null  object
 9   Value            28365 non-null  object
 10  Value Footnotes  9633 non-null   object
dtypes: object(11)
memory usage: 2.4+ MB


In [2]:
city_pop_df.shape

(28529, 11)

In [3]:
city_pop_df.head()

,Country or Area,Year,Area,Sex,City,City type,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Åland Islands,2013,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2014,5445,NaN
1,Åland Islands,2013,Total,Female,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2014,5925,NaN
2,Åland Islands,2012,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2013,5408,NaN
3,Åland Islands,2012,Total,Female,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2013,5896.5,NaN
4,Åland Islands,2011,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2012,5363.5,NaN


In [4]:
city_pop_df.dtypes

,0
Country or Area,object
Year,object
Area,object
Sex,object
City,object
City type,object
Record Type,object
Reliability,object
Source Year,object
Value,object


In [5]:
city_pop_df.describe()

,Country or Area,Year,Area,Sex,City,City type,Record Type,Reliability,Source Year,Value,Value Footnotes
count,28528,28528,28365,28365,28365,28365,28365,28365,28365,28365,9633
unique,327,194,2,3,4222,3,9,4,29,25803,157
top,Russian Federation,2010,Total,Male,Hamilton,City proper,Estimate - de jure,"Final figure, complete",2013,54738,60
freq,3028,5320,28364,14183,62,22731,12902,27698,4084,5,1318


In [6]:
city_pop_df.isnull().sum()

,0
Country or Area,1
Year,1
Area,164
Sex,164
City,164
City type,164
Record Type,164
Reliability,164
Source Year,164
Value,164


In [7]:
#examine the null count 164
null_value_df = city_pop_df[city_pop_df['Value'].isnull()]
print(null_value_df)


      Country or Area                                               Year Area  \
28364             NaN                                                NaN  NaN   
28365   footnoteSeqID                                           Footnote  NaN   
28366               1      Including armed forces stationed in the area.  NaN   
28367               2  Data refer to projections based on the 2001 Po...  NaN   
28368               3  The urban agglomeration of Buenos Aires includ...  NaN   
...               ...                                                ...  ...   
28524             159  Included in urban agglomeration of Oxnard, CA....  NaN   
28525             160  Included in urban agglomeration of San Jose, C...  NaN   
28526             161  Included in urban agglomeration of Portland, O...  NaN   
28527             162  Included in urban agglomeration of Salt Lake C...  NaN   
28528             163  Figures for male and female do not add up to t...  NaN   

       Sex City City type R

# Discovery:
there are 2 data sets in the file data and footnotes need to be seperated


---



In [8]:
#review of null values indicates footnotes are stored in area and year column,
#seperate this into a new DF
#identify the rows where 'Year' is not a valid numeric value
footnote_df = city_pop_df[pd.to_numeric(city_pop_df['Year'], errors='coerce').isnull()]

#remove these footnotes from the original dataframe
city_pop_df = city_pop_df[pd.to_numeric(city_pop_df['Year'], errors='coerce').notnull()]

#verify the results
print(f"Footnote rows separated: {len(footnote_df)}")
print(f"Remaining valid rows: {len(city_pop_df)}")

Footnote rows separated: 165
Remaining valid rows: 28364


In [9]:
#value footnotes should be remaining column with nulls
city_pop_df.isnull().sum()

,0
Country or Area,0
Year,0
Area,0
Sex,0
City,0
City type,0
Record Type,0
Reliability,0
Source Year,0
Value,0



# Data Cleaning:
Clean footnote data

---




In [10]:
footnote_df.head()

,Country or Area,Year,Area,Sex,City,City type,Record Type,Reliability,Source Year,Value,Value Footnotes
28364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28365,footnoteSeqID,Footnote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28366,1,Including armed forces stationed in the area.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28367,2,Data refer to projections based on the 2001 Po...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28368,3,The urban agglomeration of Buenos Aires includ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#clean footnote df
#drop first row, it is empty
footnote_df = footnote_df.drop(footnote_df.index[0])

#drop columns after 2, they are empty
footnote_df = footnote_df.drop(footnote_df.columns[2:11], axis=1)

#promote header row to column names
#reset index from first row drop
footnote_df.reset_index(drop=True, inplace=True)

#row 0 is now to be used as new header
footnote_df.columns = footnote_df.iloc[0]

#drop old row 2 (header row)
footnote_df = footnote_df.drop(footnote_df.index[0])

#reset index
footnote_df.reset_index(drop=True, inplace=True)

#check results
print(footnote_df.head())

0 footnoteSeqID                                           Footnote
0             1      Including armed forces stationed in the area.
1             2  Data refer to projections based on the 2001 Po...
2             3  The urban agglomeration of Buenos Aires includ...
3             4  The urban agglomeration of Tucumán-Tafí Viejo ...
4             5             Data refer to private households only.



# Data Cleaning:
Clean population data

---




In [12]:
#convert data types in population df
#source year and year to datetime
#value to float

#convert years to datetime
city_pop_df.loc['Year'] = pd.to_datetime(city_pop_df['Year'], format='%Y', errors='coerce')
city_pop_df.loc['Source Year'] = pd.to_datetime(city_pop_df['Source Year'], format='%Y', errors='coerce')

#convert the 'Value' column to float
city_pop_df['Value'] = pd.to_numeric(city_pop_df['Value'], errors='coerce')

#check result
print(city_pop_df.dtypes)

Country or Area     object
Year                object
Area                object
Sex                 object
City                object
City type           object
Record Type         object
Reliability         object
Source Year         object
Value              float64
Value Footnotes     object
dtype: object


In [13]:
city_pop_df.head()


,Country or Area,Year,Area,Sex,City,City type,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Åland Islands,2013,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2014,5445.0,NaN
1,Åland Islands,2013,Total,Female,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2014,5925.0,NaN
2,Åland Islands,2012,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2013,5408.0,NaN
3,Åland Islands,2012,Total,Female,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2013,5896.5,NaN
4,Åland Islands,2011,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2012,5363.5,NaN


In [14]:
#Cities contain state in () seperate
#function split city and state
def split_city_state(city):
    if isinstance(city, str):  #Ensure the value is a string
        if '(' in city and ')' in city:  #Check for parentheses
            city_name = city.split(' (')[0]  #Extract city
            state_name = city.split(' (')[1].strip(')')  #Extract state and remove ')'
        else:
            city_name = city  #No parentheses, so just city name
            state_name = None  #No state
    else:
        city_name = None  #Handle NaN or non-string values
        state_name = None
    return pd.Series([city_name, state_name])
#create new columns
city_pop_df[['City_Only', 'State']] = city_pop_df['City'].apply(split_city_state)

#Drop original city
city_pop_df = city_pop_df.drop('City', axis=1)

#Rename the new column
city_pop_df = city_pop_df.rename(columns={'City_Only': 'City'})

city_pop_df.head()

,Country or Area,Year,Area,Sex,City type,Record Type,Reliability,Source Year,Value,Value Footnotes,City,State
0,Åland Islands,2013,Total,Male,City proper,Estimate - de jure,"Final figure, complete",2014,5445.0,NaN,MARIEHAMN,None
1,Åland Islands,2013,Total,Female,City proper,Estimate - de jure,"Final figure, complete",2014,5925.0,NaN,MARIEHAMN,None
2,Åland Islands,2012,Total,Male,City proper,Estimate - de jure,"Final figure, complete",2013,5408.0,NaN,MARIEHAMN,None
3,Åland Islands,2012,Total,Female,City proper,Estimate - de jure,"Final figure, complete",2013,5896.5,NaN,MARIEHAMN,None
4,Åland Islands,2011,Total,Male,City proper,Estimate - de jure,"Final figure, complete",2012,5363.5,NaN,MARIEHAMN,None


In [15]:
#filter US only, most recent year
us_df = city_pop_df[city_pop_df['Country or Area'] == 'United States of America']

us_latest_year_df = us_df.loc[us_df.groupby(['City', 'State', 'Sex'])['Year'].idxmax()]

#check results
us_latest_year_df.head(10)


,Country or Area,Year,Area,Sex,City type,Record Type,Reliability,Source Year,Value,Value Footnotes,City,State
27136,United States of America,2010,Total,Female,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,57956.0,129130,Abilene,TX
26854,United States of America,2010,Total,Male,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,59107.0,129130,Abilene,TX
27137,United States of America,2010,Total,Female,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,102853.0,129130,Akron,OH
26855,United States of America,2010,Total,Male,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,96257.0,129130,Akron,OH
27138,United States of America,2010,Total,Female,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,280746.0,129130,Albuquerque,NM
26856,United States of America,2010,Total,Male,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,265106.0,129130,Albuquerque,NM
27139,United States of America,2010,Total,Female,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,72704.0,129130,Alexandria,VA
26857,United States of America,2010,Total,Male,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,67262.0,129130,Alexandria,VA
27140,United States of America,2010,Total,Female,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,61163.0,129130,Allentown,PA
26858,United States of America,2010,Total,Male,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,56869.0,129130,Allentown,PA


# Pull in Additional Data
Geolocation data csv obtained from https://simplemaps.com/data/world-cities  ... creative commons attribution


---



In [16]:
geolocation_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Population data/worldcities.csv',
usecols=['city_ascii','state', 'lat', 'lng', 'country'])
#rename city_ascii to city
geolocation_df.rename(columns={'city_ascii': 'city'}, inplace=True)

#need to convert State name to abbreviation
def convert_state_to_abbr(state_name):
    if isinstance(state_name, str):
        #Special case for Washington DC
        if state_name in ["District of Columbia", "Washington DC"]:
            return "DC"
        else:
            state = us.states.lookup(state_name)
            if state:
                return state.abbr
    return state_name  #if state name is not found or is NaN, return the original value

#apply to state column
geolocation_df['state_abbr'] = geolocation_df['state'].apply(convert_state_to_abbr)


geolocation_df.head()

,city,lat,lng,country,state,state_abbr
0,Tokyo,35.6897,139.6922,Japan,Tōkyō,Tōkyō
1,Jakarta,-6.1750,106.8275,Indonesia,Jakarta,Jakarta
2,Delhi,28.6100,77.2300,India,Delhi,Delhi
3,Guangzhou,23.1300,113.2600,China,Guangdong,Guangdong
4,Mumbai,19.0761,72.8775,India,Mahārāshtra,Mahārāshtra


In [17]:
#merge data
merged_US_df = pd.merge(us_latest_year_df, geolocation_df,
                       left_on=['City','State'],
                       right_on=['city', 'state_abbr'],
                       how='left')

#drop uneeded columns
merged_US_df = merged_US_df.drop(['city', 'state', 'state_abbr', 'country'], axis=1)

merged_US_df.head()

,Country or Area,Year,Area,Sex,City type,Record Type,Reliability,Source Year,Value,Value Footnotes,City,State,lat,lng
0,United States of America,2010,Total,Female,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,57956.0,129130,Abilene,TX,32.4543,-99.7384
1,United States of America,2010,Total,Male,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,59107.0,129130,Abilene,TX,32.4543,-99.7384
2,United States of America,2010,Total,Female,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,102853.0,129130,Akron,OH,41.0798,-81.5219
3,United States of America,2010,Total,Male,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,96257.0,129130,Akron,OH,41.0798,-81.5219
4,United States of America,2010,Total,Female,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,280746.0,129130,Albuquerque,NM,35.1054,-106.6465


In [18]:
merged_US_df.isnull().sum()

,0
Country or Area,0
Year,0
Area,0
Sex,0
City type,0
Record Type,0
Reliability,0
Source Year,0
Value,0
Value Footnotes,0


In [19]:
# Filter rows where 'lat' is null
null_lat_df = merged_US_df[merged_US_df['lat'].isnull()]

# Display the rows where 'lat' is null
null_lat_df.head(20)

,Country or Area,Year,Area,Sex,City type,Record Type,Reliability,Source Year,Value,Value Footnotes,City,State,lat,lng
54,United States of America,2010,Total,Female,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,103981.0,129130,Boise City,ID,NaN,NaN
55,United States of America,2010,Total,Male,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,101690.0,129130,Boise City,ID,NaN,NaN
226,United States of America,2010,Total,Female,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,55369.0,129130,High Point City,NC,NaN,NaN
227,United States of America,2010,Total,Male,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,49002.0,129130,High Point City,NC,NaN,NaN
274,United States of America,2010,Total,Female,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,150212.0,129130136,Lexington-Fayette,KY,NaN,NaN
275,United States of America,2010,Total,Male,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,145591.0,129130136,Lexington-Fayette,KY,NaN,NaN
298,United States of America,2010,Total,Female,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,66683.0,129130,McKinney City,TX,NaN,NaN
299,United States of America,2010,Total,Male,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,64434.0,129130,McKinney City,TX,NaN,NaN
312,United States of America,2010,Total,Female,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,57018.0,129130,Midland City,TX,NaN,NaN
313,United States of America,2010,Total,Male,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,54129.0,129130,Midland City,TX,NaN,NaN


# Discovery, More data cleaning needed:
Merge was not entirely successful due to number of nulls in merged data.
Likely caused by differences in city between 2 data sources.

In [20]:
#dictionary for corrections because there are few
city_corrections = {
    'Boise City': 'Boise',
    'High Point City': 'High Point',
    'Lexington-Fayette': 'Lexington',
    'McKinney City': 'McKinney',
    'Midland City': 'Midland',
    'Nashville-Davidson': 'Nashville',
    'Palm Bay City': 'Palm Bay',
    'Urban Honolulu': 'Honolulu',
    'Victorville City': 'Victorville',
    'WASHINGTON': 'Washington'
}
#function to apply corrections
def correct_city_name(city_name):
    if city_name in city_corrections:
        return city_corrections[city_name]
    return city_name  # Return the original name if it's not in the dictionary
#apply function
us_latest_year_df['City'] = us_latest_year_df['City'].apply(correct_city_name)

#merge data again
merged_US_df = pd.merge(us_latest_year_df, geolocation_df,
                       left_on=['City','State'],
                       right_on=['city', 'state_abbr'],
                       how='left')

#drop uneeded columns
merged_US_df = merged_US_df.drop(['city', 'state', 'state_abbr', 'country'], axis=1)

#check results
merged_US_df.isnull().sum()


,0
Country or Area,0
Year,0
Area,0
Sex,0
City type,0
Record Type,0
Reliability,0
Source Year,0
Value,0
Value Footnotes,0


In [21]:
merged_US_df.head()

,Country or Area,Year,Area,Sex,City type,Record Type,Reliability,Source Year,Value,Value Footnotes,City,State,lat,lng
0,United States of America,2010,Total,Female,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,57956.0,129130,Abilene,TX,32.4543,-99.7384
1,United States of America,2010,Total,Male,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,59107.0,129130,Abilene,TX,32.4543,-99.7384
2,United States of America,2010,Total,Female,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,102853.0,129130,Akron,OH,41.0798,-81.5219
3,United States of America,2010,Total,Male,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,96257.0,129130,Akron,OH,41.0798,-81.5219
4,United States of America,2010,Total,Female,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,280746.0,129130,Albuquerque,NM,35.1054,-106.6465


#Visualize data

In [22]:
# Pivot the data to get male and female populations for each city
pivot_df = merged_US_df.pivot_table(index=['City', 'State', 'lat', 'lng'], columns='Sex', values='Value').reset_index()

# Calculate the total population for each city
pivot_df['Total Population'] = pivot_df['Male'] + pivot_df['Female']

# Merge the total population back into the original DataFrame
merged_total_df = pd.merge(merged_US_df, pivot_df[['City', 'State', 'Total Population', 'Male', 'Female']], on=['City', 'State'], how='left')

# Create a new column for tooltip text with total population, male, and female populations
merged_total_df['tooltip_text'] = (
    merged_total_df['City'] + '<br>' +
    'Total Population: ' + merged_total_df['Total Population'].astype(str) + '<br>' +
    'Male Population: ' + merged_total_df['Male'].astype(str) + '<br>' +
    'Female Population: ' + merged_total_df['Female'].astype(str) + '<br>' +
    'Year: ' + merged_total_df['Year'].astype(str)
)

# Set the color scale based on the total population
cmin_value = merged_total_df['Total Population'].min() + 35000  # Adjusted minimum for better gradient
cmax_value = merged_total_df['Total Population'].max() - 7000000  # Adjusted maximum for better gradient

# Build map with plotly
fig = go.Figure(data=go.Scattergeo(
    lon=merged_total_df['lng'],  # Longitude values
    lat=merged_total_df['lat'],  # Latitude values
    text=merged_total_df['tooltip_text'],  # Tooltip text with total, male, and female populations
    mode= 'markers',  # Display markers
    marker=dict(
        size=8,  # Size of the marker
        color=merged_total_df['Total Population'],  # Use total population for color shading
        colorscale='Plasma',  # Color scale for visualizing the total population
        cmin=cmin_value,  #
        cmax=cmax_value,
        colorbar=dict(title="Total Population"),
        opacity=0.7,  # Marker transparency
    ),
))

# Update layout for USA map and tooltips
fig.update_layout(
    title='Major City Population by Sex and Total<br>zoom in for overlapping points<br>(Hover for details)',  # Title of the map
    geo_scope='usa',  # Set the scope to USA
    geo=dict(
        showland=True,  # Show landmasses
        landcolor="lightgray",  # Set land color
    ),
    width=1600,  # Adjust the width of the figure
    height=1000,  # Adjust the height of the figure
)

# Display the map
fig.show()
